# Supervised Learning

In [ ]:
reset

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline

&nbsp;

# 1 OLS Regression with `statsmodels`

* to install `conda install statsmodels` or `pip install statsmodels`    
* uses package <a href = 'http://patsy.readthedocs.io/en/latest/'>pasty</a> which allows passing **R** style formulas.  
* contains a large number of modules for statistical testing and model estimation <a href= 'http://www.statsmodels.org/stable/py-modindex.html'>Link</a>

## 1.1 data cleaning

In [ ]:
filepath = 'data/homes.csv'
dataset = pd.read_csv(filepath, header = 0, sep = ',')

dataset.head(10)

In [ ]:
dataset.shape

In [ ]:
dataset.dtypes

In [ ]:
dataset.id = dataset.id.astype('category')
dataset.waterfront = dataset.waterfront.astype('category')
dataset.view = dataset.view.astype('category')
dataset.grade = dataset.grade.astype('category')
dataset.condition = dataset.condition.astype('category')
dataset = dataset.drop(['zipcode'], axis = 1)
dataset = dataset.drop(['sqft_living15'], axis = 1)
dataset = dataset.drop(['sqft_lot15'], axis = 1)

* this dataset is collected in 2015 and the latest year for a home build or renovated is 2015.      
* for the columns `yr_built` and `yr_renovated` to keep them as numeric variables it is better to convert the values in these column in the form of an offset from 2015. 

In [ ]:
dataset.yr_built = [2015 - yr for yr in dataset.yr_built]

dataset.yr_renovated = [2015 - yr if yr != 0 else yr for yr in dataset.yr_renovated]

&nbsp;

<span style='color:blue'>in two steps using `map()` and `lambda()` operator convert the date column into a numpy `datetime64`   
how do you think you can go about this task given a date variable with `20141013T000000` format?</span>

In [ ]:
#skipped code

In [ ]:
#skipped code
#new_date = 

In [ ]:
dataset.date = new_date

In [ ]:
dataset.head()

&nbsp;
* convert the `id` column a dataframe index

In [ ]:
dataset = dataset.set_index(['id'])

dataset.head()

&nbsp;

<span style="color:blue">rearrange the columns in the dataset to push the four cateorical variables to the very end.</span>    


* in addition purge the last two columns

In [ ]:
#skipped code

In [ ]:
#skipped code
#new_cols 

In [ ]:
dataset.head()

In [ ]:
dataset.dtypes

&nbsp;

## 1.2 pair plots and correlations

In [ ]:
# this takes a long time to process
#sns.pairplot(dataset.iloc[:,1:11], diag_kind = 'kde')

In [ ]:
corr = dataset.iloc[:,1:11].corr()

mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

sns.set(style="white") # options include white, dark, whitegrid, darkgrid, ticks
sns.heatmap(corr, mask = mask)

* there is a strong positive correlation between `sqft_above` (Square footage of house apart from basement) and `sqrt_living`
(Square footage of the living room) at 0.876597

In [ ]:
dataset = dataset.drop(['sqft_above'], axis = 1)

&nbsp;

## 1.3 model estimation and diagnostics

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import statsmodels.formula.api as smf

`train_test_split(*array, test_size = .25, train_size, random_state, shuffle = True)`

test_size/train_size: 
        * float => proportion of data      
        * int => absolute number of observations


In [ ]:
d_train, d_test = train_test_split(dataset, test_size = 0.2, random_state = 43)

In [ ]:
d_train.shape, d_test.shape

&nbsp;


* we shall perform a **hedonic regression** using the homes dataset after cleaning.

* for this we do not require the `date` column

In [ ]:
d_train = d_train.drop(['date'], axis = 1)
d_test = d_test.drop(['date'], axis = 1)

&nbsp;

statsmodels requires that the function call be passed in be a string in the format `output ~ input_! + input_2 + ... input_n`

In [ ]:
function_call = 'price ~ ' + ' + '.join(d_train.columns[1:].values)

In [ ]:
function_call

In [ ]:
lmod = smf.ols(function_call , data = d_train)

In [ ]:
results = lmod.fit()
print(results.summary())

In [ ]:
print('F statistic:   ',results.fvalue)
print('F stat pvalue: ',results.f_pvalue)
print('r squared:     ',results.rsquared)
print('r squared adj: ',results.rsquared_adj)
print('residuals df:  ',results.df_resid)
print('SST:           ',results.mse_total*(results.df_resid + results.df_model))
print('SSE:           ',results.mse_resid*results.df_resid)
print('SSM:           ',results.mse_model*results.df_model)
print('resid std err: ',results.mse_resid**.5)

In [ ]:
sns.set(style="darkgrid")
plt.figure(figsize=(10,7))
plt.plot(np.arange(d_train.shape[0]),results.resid, '.')

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(results.fittedvalues, results.resid, '.')

## 1.4 model Validation 

In [ ]:
lmod_pred = results.predict(d_test)

In [ ]:
mse_prediction = 1/(d_test.shape[0])*np.sum((lmod_pred - d_test.price)**2)
mse_prediction

<span style='color:blue'>what do you think about the value of MSE ?</span>

&nbsp;

we can also calculate the `mean_squared_error` using the method with the same name from `sklearn.metrics`.

In [ ]:
mean_squared_error(y_true = d_test.price, y_pred = lmod_pred)

In [ ]:
scipy.stats.kstest(results.resid, 'norm')

In [ ]:
plt.figure(figsize=(10,7))
sns.distplot(results.resid, bins = 100, fit = scipy.stats.norm)

In [ ]:
sm.qqplot(results.resid, dist = scipy.stats.norm ,fit = True, line='45')
plt.axis([-5,5,-15,15])

&nbsp;

<span style='color:blue'>can we do anything to make the residuals Gaussian ?</span>

In [ ]:
#skipped code


In [ ]:
lmod_1 = smf.ols(function_call_1, data = d_train)

results_1 = lmod_1.fit()
print(results_1.summary())

&nbsp;

&nbsp;


In [ ]:
print('F statistic:   ',results_1.fvalue)
print('F stat pvalue: ',results_1.f_pvalue)
print('r squared:     ',results_1.rsquared)
print('r squared adj: ',results_1.rsquared_adj)
print('residuals df:  ',results_1.df_resid)
print('SST:           ',results_1.mse_total*(results.df_resid + results.df_model))
print('SSE:           ',results_1.mse_resid*results.df_resid)
print('SSM:           ',results_1.mse_model*results.df_model)
print('resid std err: ',results_1.mse_resid**.5)

`F statistic:    1151.65066416
F stat pvalue:  0.0
r squared:      0.651343773911
r squared adj:  0.650778199823
residuals df:   17261.0
SST:            4789.71978586
SSE:            1669.96562456
SSM:            3119.7541613
resid std err:  0.311043256005`

&nbsp;

<span style='color:blue'>first plot `results.resid` followed by `results_1.resid` vs `results_1.fittedvalues` (similar to the previous plots)      
what do you think about the residuals? </spab>

In [ ]:
#different plotting package
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

&nbsp;

plot results.resid for a quick comparison

In [ ]:
p = figure(plot_width = 800, plot_height = 400)
p.scatter(range(d_train.shape[0]), results.resid, color = 'purple')
show(p)

In [ ]:
p = figure(plot_width = 800, plot_height = 400)
p.scatter(range(d_train.shape[0]), results_1.resid, color = 'purple')
show(p)

In [ ]:
p = figure(plot_width = 700, plot_height = 400)
p.scatter(results_1.fittedvalues, results_1.resid, color = 'purple')
show(p)

&nbsp;

* predicting the test set

In [ ]:
lmod_pred_1 = results_1.predict(d_test)

In [ ]:
mse_prediction = 1/(d_test.shape[0])*np.sum((lmod_pred_1 - np.log(d_test.price))**2)
mse_prediction

In [ ]:
np.exp(mse_prediction)

In [ ]:
scipy.stats.kstest(results_1.resid, 'norm')

In [ ]:
plt.figure(figsize=(10,7))
sns.distplot(results_1.resid, bins = 100, fit = scipy.stats.norm, color = 'purple')

In [ ]:
fig = sm.qqplot(results_1.resid, dist = scipy.stats.norm ,fit = True, line='45')
plt.show()

&nbsp;

* can we infer anything from the Jarque-Bera test of normality ?

In [ ]:
scipy.stats.jarque_bera(results_1.resid)

* the Jarque-Bera test is a test for normality that is based on sample skewness and kurtosis with     
`H0: the distribution is normal`       
`H1: the distribution is not normal`.   



* the calculated statistic $JB = \frac{N}{6} \bigg(S^2 + \frac{(K-3)^2}{4} \bigg)$   
* with a sample size > 2000 the test statistic is compared to a $\chi^2$ distribution with 2 degrees of freedom. 

In [ ]:
# this is what a chi-square distribution with df=2 looks like
np.random.seed(40)
chisq_sim = np.random.chisquare(2, 20000)
sns.distplot(chisq_sim, bins = 1000)

* despite the fact that the Jarque-Bera test is conclusive, and there is enough evidence to reject the null, the JB statistic went from 266387.882 to 121.22. However for a p-value to not reject the Null the JB statistic needs to approximately between 0 and 4.

&nbsp;

# 2 OLS Regression using `scikit-learn`


* scikit learn does not ingest categorical variables passed into the method input. instead all categorical variable need to be processed prior to running the model estimate. 

In [ ]:
dataset.dtypes, dataset.shape

## 2.1 encoding categorical variables

* `scikit-learn` does not automatically handle categorical variables.   
* in order to use the `LinearRegression` module and other modules we have to encode all the categorical variable in dummy coding.   

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

`LabelEncoder` is the method that converts categorical non-numeric variables into numeric representation   
`OneHotEncoder` is the method that converts numeric variables into dummy variable  
&nbsp;

In [ ]:
[np.unique(dataset.iloc[:,i], return_counts = True) for i in range(10,14)]

we have two catagorical variables starting at level `0` and two starting at level `1`

hence these two variables require the use of `LabelEncoder` first.

&nbsp;

the following series of steps are often analogous for different methods of scikit-learn

1- assign the method to a name space (creating an instance of that method)     
**set the paramteres and arguments of the new instance method in this step**

In [ ]:
lbc = LabelEncoder()

enc = OneHotEncoder()

2- apply the instance method `.fit` to ingest the data using the method

In [ ]:
lbc_condition = lbc.fit(dataset.condition)

3 - apply the instance method `.transform` to interpret the results and produce an output. 

In [ ]:
lbc_condition = lbc.transform(dataset.condition)

In [ ]:
lbc_condition

&nbsp;

steps 3 and 4 can often be combined together in a single expression

In [ ]:
lbc_condition = lbc.fit(dataset.condition).transform(dataset.condition).reshape((-1,1))

In [ ]:
np.unique(lbc_condition, return_counts = True)

In [ ]:
lbc_grade = lbc.fit(dataset.grade).transform(dataset.grade).reshape((-1,1))

In [ ]:
cat_array = np.hstack([np.array(dataset.waterfront).reshape((-1,1)), 
                        np.array(dataset.view).reshape((-1,1)), 
                        lbc_condition, lbc_grade])

In [ ]:
cat_array

In [ ]:
fit_obj = enc.fit(categorical_array)

cat_vars = fit_obj.transform(categorical_array).toarray()

In [ ]:
cat_vars

In [ ]:
cat_vars.shape

&nbsp;

this is good.    

however for the regression problem we the first dummy variable from every category to be removed in order to allow for correct calculation of an intercept 

how can we go about it ?

display the instance methods and attributed of the `fit_obj` calculated above

In [ ]:
[i for i in dir(fit_obj) if not i.startswith('_')]

In [ ]:
fit_obj.feature_indices_

these indeces represent the first or every group of dummy variables corresponding to the 4 variables.      
the last index is the total number of columns overall. 

In [ ]:
['view_'+str(i) for i in range(1,5)]

In [ ]:
names_ = ['waterfront1']+\
         ['view_'+str(i) for i in range(1,5)]+\
         ['con_'+str(i) for i in range(2,6)]+\
         ['g_'+str(i) for i in range(3,14)]
names_ 

In [ ]:
cat_var_clean = pd.DataFrame(np.delete(cat_vars, fit_obj.feature_indices_[:-1], 1), columns = names_)
cat_var_clean.head()

In [ ]:
cat_var_clean.shape

* drop the original categorical variables to substitute with the dummy data frames. 

In [ ]:
dataset_skl = dataset.drop(['date'], axis = 1).reset_index(drop = True).copy()

#both tables need to have the same index. 
#either drop index of dataset or assign its index value to vat_var_clean

dataset_skl = dataset_skl.drop(['waterfront', 'view', 'grade', 'condition'], axis = 1)

In [ ]:
dataset_skl = pd.concat([dataset_skl, cat_var_clean], axis = 1)

In [ ]:
dataset_skl.head()

&nbsp;


## 2.2 Model estimation and diagnistics 

In [ ]:
# use the same seed to retrieve an indentical train/test split.
d_train_skl, d_test_skl = train_test_split(dataset_skl, test_size = 0.2, random_state = 43)

In [ ]:
from sklearn import linear_model, preprocessing

`sklearn.linear_model` takes an argument for the input and an argument for the output(s) 

In [ ]:
train_x = d_train_skl.iloc[:,1:]
train_y = np.log(d_train_skl.iloc[:,0])

&nbsp;


recall the steps from earlier: 

1- create an instance of the method and set the parameters and arguments

In [ ]:
ols = linear_model.LinearRegression(fit_intercept = True, normalize = False, copy_X = True)

2- apply the instsance method `.fit` using the variables or input and outpute 

In [ ]:
ols_fit = ols.fit(X = train_x, y = train_y)

In [ ]:
ols.coef_

In [ ]:
results_1.params[0], ols.intercept_

&nbsp;

we can compare the coefficients from statsmodels `results` object and sklearn coefficients. 

In [ ]:
a = pd.DataFrame(results_1.params, columns = ['statsmodels']).sort_values(by = 'statsmodels').drop(['Intercept'], axis = 0)
b = pd.DataFrame(ols_fit.coef_, columns = ['sklearn']).sort_values(by = 'sklearn').set_index(a.index)
df = pd.concat([b,a], axis = 1).sort_index()
df

&nbsp;

* in order to obtain the fittedvalues and residuals we have to calculate these manually. 

$$\begin{bmatrix} fitted\ val1 \\ fitted\ val2 \\ . \\ . \\ . \\ fitted\ valN \end{bmatrix} = intercept + 
\begin{bmatrix} x_{1,1} & x_{1,2} & . . . & x_{1,m-1} & x_{1,m} \\ 
                x_{2,1} & \ddots & . . . & x_{2,m-1} & x_{2,m} \\ 
                . & . & \ddots & . & . \\ 
                . & . & . & \ddots  & . \\ 
                x_{n-1,1} & x_{n-1,2} & . . . & x_{n-1,m-1} & x_{n-1,m} \\
                x_{n,1} & x_{n,2} & . . . & x_{n,m-1} & x_{n,m} 
                \end{bmatrix} \times \begin{bmatrix} \theta_{bathrooms} \\ \theta_{bedrooms} \\ . \\ \theta_{grade[T.3]} \\ . \\ \theta_{yr\ renovated} \end{bmatrix}$$
                
&nbsp;                

$$Residuals = Output - Fitted\ Values$$

In [ ]:
ols_fittedvalues = np.dot(train_x, np.array(ols.coef_).T)

In [ ]:
ols_resids = train_y.values - ols_fittedvalues

In [ ]:
p = figure(plot_width = 800, plot_height = 400)
p.scatter(range(train_x.shape[0]), ols_fittedvalues, color = 'peachpuff')
show(p)

In [ ]:
p = figure(plot_width = 800, plot_height = 400)
p.scatter(ols_fittedvalues, ols_resids, color = 'peachpuff')
show(p)

* the instance method `.score` of the fit object `ols` returns the coefficient of determination $R^2$

In [ ]:
ols_rsq = ols.score(X = train_x, y = train_y)
ols_rsq

similarly `fit.rank_` returns the degrees of freedom 

In [ ]:
deg_freedom = ols_fit.rank_
deg_freedom

&nbsp; 

$adj\_r.squared$ can be calculated manually 

In [ ]:
n, k = train_x.shape[0], len(train_x.columns.values)
n , k

$adj\_r.squared = 1 - \bigg[\frac{(1 - R^2)(n-1}{n-k-1}\bigg] $

In [ ]:
ols_adj_rsq = 1 - ((1 - ols_rsq)*(n-1)/(n-k-1))
ols_adj_rsq

&nbsp;


## 2.3 Model validation 

In [ ]:
test_x = d_test_skl.iloc[:,1:]
test_y = np.log(d_test_skl.iloc[:,0])

&nbsp;

3- instead of `.transform()` the method `.predict()` takes the inputs of the test set and calculates a predicted output 

In [ ]:
ols_pred = ols_fit.predict(test_x)

In [ ]:
mean_squared_error(y_true = test_y, y_pred = ols_pred)

&nbsp;

&nbsp;

### Addendum

instead of using `OneHotEncoder()` the method `pandas.get_dummies()` allows us to convert a categorical variable into dummy variables and allows us to drop the first column by setting an argument within the method call.     
 
 
since this is a pandas method it preserves the index of the original dataframe   

using <U>dataset</U>   

In [ ]:
pd_waterfront = pd.get_dummies(dataset.waterfront, drop_first = True)
pd_view = pd.get_dummies(dataset.view, drop_first = True)
pd_condition = pd.get_dummies(dataset.condition, drop_first = True)
pd_grade = pd.get_dummies(dataset.grade, drop_first = True)

In [ ]:
pd_condition.head()

In [ ]:
dataset_d = pd.concat([dataset.iloc[:,1:10], pd_waterfront, pd_view, pd_condition, pd_grade], axis = 1)

In [ ]:
dataset_d.columns = dataset.columns[1:10].tolist() + names_

In [ ]:
dataset_d.head()